<div style="padding:30px; color: white; background-color: #0071CD">
<center>
    <img src="img/logoub.jpeg"></img>
<p>
<h1>Algorítmica Avanzada</h1>
<h2>Problemas Enumerativos</h2>
</center>
</p>
</div>

<div class="alert alert-info">
<center>
  <h1>Backtracking</h1>
</center>
</div>

<div class="alert alert-success" style="width:90%; margin:0 auto;">
<h2><p>0. Cartas</p></h2>
<p>
Supongamos que tenemos un conjunto de parejas de cartas del mismo valor. En particular, tenemos parejas de cartas hasta N. Por ejemplo, para $N=3$ tendríamos una pareja de 1s, una de 2s y una de 3s (un total de 6 cartas). Lo que queremos es encontrar la manera de ordenarlas para que entre las dos cartas de valor $n_i$ haya exactamente $n_i$ cartas. En el ejemplo con $N=3$ tendríamos la siguiente solución:
</p>
<center> <b>   3 - 1 - 2 - 1- 3 - 2 </b></center>
<p>
Podemos observar que entre los 3 hay tres cartas, entre los 2 hay dos y entre los 1 solo hay una. Dependiendo del tamaño de N el problema podría no tener solución. Por ejemplo $N=5$ o $N=6$. Se pide la implementación de un algoritmo que, mediante backtracking, devuelva una lista con la configuración encontrada dado un valor N. En caso de no haber solución, debe devolver una lista vacía.
</p> <p>
Es importante destacar que tendréis que prestar especial atención a la complejidad de vuestra solución, dado que el problema crece de forma no polinómica y para un valor $N>12$ podría tardar horas. Hay diferentes aproximaciones para resolverlo. Bien hecho, $N=12$ debería tardar escasos segundos. Si haces varias versiones, puedes entregarlas junto con la comparativa de complejidad correspondiente.</p>
</div>

In [1]:
def compute(n):
    """
    Computes the possible combinations
    of positions for every card.
    
    Parameters
    ----------
        n, int
        
    Returns
    -------
        dict
            Possible combinations
    """
    
    sol = {}
    
    for i in range(n, 0, -1):
        sol[i] = []
        
        for j in range((2 * n) - i - 1):
            x, y = j, j + i + 1
            
            sol[i].append((x, y))  
    
    return sol

compute(3)

{3: [(0, 4), (1, 5)],
 2: [(0, 3), (1, 4), (2, 5)],
 1: [(0, 2), (1, 3), (2, 4), (3, 5)]}

In [2]:
def naive_deck(n, solutions={}, selected=[]):
    """
    Naive implementation.
    Recursive and slow af.
    For n > 12, if takes a while...
    
    Parameters
    ----------
        n, int
        
        solutions, dict 
            Rec. parameter
        selected, list
            Rec. parameter
        
    Returns
    -------
        list
            Card ordering, or
            empty if can not.
    """
    
    if n == 0:
        return selected if all(selected) else []
    
    for x, y in solutions[n]:
        ss = selected.copy()
        
        if not ss[x] and not ss[y] and n not in ss:            
            ss[x], ss[y] = n, n
            
            # print(n, (x, y), ss)
            _s = naive_deck(n - 1, solutions=solutions, selected=ss)
            
            if _s and len(_s) > 0:
                return _s
    
    return selected if all(selected) else []

n = 3
naive_deck(n, solutions=compute(n), selected=[False for _ in range(2 * n)])

[3, 1, 2, 1, 3, 2]

In [3]:
import numpy as np

def deck(n):
    """
    Algorithm to compute the ordering
    of n pairs of cards with the given
    conditions.
    
    Parameters
    ----------
        n, int
        
    Returns
    -------
        list
            Deck ordering or
            empty list if do not exist
    """
    
    # Init the (2 * n) x n matrix 
    # with 0 values (initial states).
    
    m = np.array([[ 0 for _ in range(2 * n)] for _ in range(n) ], dtype=np.int)
    i, error = 0, False
    
    while i < n:
        p = n - i
        
        # Initial case
        
        if i == 0:
            
            # The _break variable will be
            # used in this algorithm to check
            # if we selected a pair or not.
            
            _break = False
            
            for j in range((2 * n) - p - 1):
                
                # Only select when the current 
                # position is a zero (initial state)
                
                if m[i][j] == 0:
                    
                    # Mark the selected column (j)
                    # with the pivot, in this case n.
                    # And set _break to True,
                    # as we found a candidate, 
                    # and break the loop.
                    
                    m[i:,j], m[i:,j + p + 1] = p, p
                    _break = True
                    
                    break
            
            # If we could not find a candidate
            # we break the main while loop with
            # an error code, as we could not find
            # a valid position for the pair (n, n).
            
            if not _break:
                error = True
                break
                
            # If we could find a candidate
            # continue in the next iteration
                
            i += 1
            continue
        
        
        # "Normal" case
        
        _break = False
        
        # Loop over all possible 
        # positions for the current pivot
        
        for j in range((2 * n) - p - 1):
            
            # If we find anything else from a zero
            # (used states)
            # then it's not a valid position,
            # so we continue to the next candidate.
            
            if m[i][j] != 0 or m[i][j + p + 1] != 0:
                continue
            
            # If we found a candidate, we mark
            # the pair of columns with the pivot,
            # but from the i-th row to the last.
            # We set _break = True like before.
            
            m[i:,j], m[i:,j + p + 1] = p, p
            _break = True
            
            break
        
        # If we could not find a valid candidate for p
        # then some pivot in the upper rows is incorrect.
        
        if not _break:
            
            # We find the indices of the upper pivot
            # and mark the FIRST one with a -1 (used state)
            # and the other with a 0 (reset the state to initial).
            # See that from the first candidate column
            # we only mark the (i-1)-th row, so the
            # next pivots are not affected by this change.
            
            _indexes = np.argwhere(m[i - 1] == p + 1)
            
            m[i - 1,_indexes[0]] = -1
            m[i:,_indexes[0]] = 0
            m[i - 1:,_indexes[1]] = 0
            
            # Go back to the upper row
            
            i -= 1
            continue
        
        # If all OK, we continue to the following row
        
        i += 1
    
    # If there was an error, the order
    # do not exist, so we return an empty list
        
    return list(m[-1]) if not error else list()

In [4]:
def check_deck(n, d):
    """
    Checks if a given deck
    ordering is correct, i.e. it
    satisfies the given conditions.
    
    Parameters
    ----------
        n, int
        d, list
        
    Returns
    -------
        boolean
    """
    
    if len(d) == 0:
        return None
    
    arr = np.array(d)
    
    for i in range(1, n + 1):
        x, y = np.argwhere(arr == i).flatten()
        
        if x + i + 1 != y:
            return False
        
    return True

In [5]:
for n in range(1, 41):
    sol = deck(n)
        
    if len(sol) > 0:
        if check_deck(n, sol):
            print(f"deck({n:2d}): OK!")

deck( 3): OK!
deck( 4): OK!
deck( 7): OK!
deck( 8): OK!
deck(11): OK!
deck(12): OK!
deck(15): OK!
deck(16): OK!
deck(19): OK!
deck(20): OK!
deck(23): OK!
deck(24): OK!
deck(27): OK!
deck(28): OK!
deck(31): OK!
deck(32): OK!
deck(35): OK!
deck(36): OK!
deck(39): OK!
deck(40): OK!


## Analiza el coste de tu/s implementación/es:

### `naive_deck`

Algoritme gairebé directe. La complexitat és 
$$O(n) = (n-1) \cdots (2n - 2) = \frac{2(n - 1)!}{(n - 2)}$$

### `deck`

Versió optimitzada, inspirada en l'algoritme de backtracking utilitzat per resoldre el problema de les $N$ reines.

La complexitat és molt menor que la de l'algoritme anterior, ja que es rejecten molts casos de manera prou eficient.

$$O(n) = \cdots$$


<div class="alert alert-info">
<center>
  <h1>Ramificación y poda</h1>
</center>
</div>


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>1. Mochila</p></h2>
  
   <p> En esta primera sección trabajaremos con el problema de la mochila que ya vimos en problemas Greedy y PD. Refrequemos un poco (por si aún no lo tenéis claro):
</p>
    Nos encontramos en una habitación en la que hay $N$ objetos con pesos $w_1, w_2, w_3 ... w_N$ y tenemos una mochila que puede soportar una carga máxima de $W$. En este caso se pide que realicéis una implementación de Ramificación y Poda para resolver el problema de la mochila con valor:
<br><br>
 **mochila_valor:** Cada objeto tendrá asignado un valor $v_1, v_2, v_3 ... v_N$. Buscamos llenar la mochila maximizando el valor total de la mochila pero sin superar la capacidad máxima. Es decir queremos encontrar la combinación de objetos $b$ tal que $\arg_{b} \max{\sum_{i=0}^{N}{v_i · b_i}}$ con la condición de que $\sum_{i=0}^{N}{(w_i · b_i)}\leq W$.
     <br><br>
Trabajaremos únicamente con la versión de la mochila en la que tenemos una cantidad limitada $c_1, c_2, c_3 ... c_N$ de objetos.</div>


In [6]:
import random as rd

# Genera un conjunto de objetos de prueba en forma de una lista de listas, 
# cada una de las cuales representa un elemento [peso, valor, cantidad]

def random_objects():
    objects = [
        [p,v,c] 
        for p,v,c in zip(
            rd.sample(range(1, 50), k=10), 
            rd.sample(range(1, 100), k=10),
            rd.sample(range(1, 15), k=10))
    ]
    
    return objects

In [7]:
def knapsack_branch(D, W):
    """
    Kanpsack problem solutions using 
    a branch and bound approach.
    
    Parameters
    ----------
        D, list
            List of the objects in the form
                [ ... [ weight, value, quant. ] ... ]
        W, int
            Capacity of the backpack
    
    Returns
    -------
        list
    """
    return list()


In [8]:
rd.seed(0) 

D, W = random_objects(), 150
print(D, W)

print(knapsack_branch(D, W))

[[25, 75, 12], [27, 28, 13], [3, 65, 10], [17, 18, 3], [33, 37, 5], [32, 97, 2], [26, 13, 9], [20, 80, 7], [31, 33, 6], [23, 69, 11]] 150
[]


### Analiza el coste de tu implementación. Comparalo además con el coste de las implementaciones greedy y dinámica. Compara también la optimalidad de la solución en los tres casos.

bla


<div class="alert alert-success" style="width:90%; margin:0 auto;">
  <h2><p>2 - Orden de Ensamblaje</p></h2>
  
  <div class="row">
  <div>
  <p style="text-align: justify; padding-right: 30px">
La construcción de un artefacto requiere ensamblar $M$ piezas que identiﬁcamos con números entre $1$ y $M$. El coste de ensamblar la pieza $i$ depende del número de piezas ya ensambladas. Los costes, que son valores positivos, se nos proporcionan en una matriz $C$. La celda $c_{ij}$ es el coste de ensamblar la pieza $i$ cuando ya se han ensamblado $j$ piezas. Deseamos calcular el orden de ensamblado de menor coste total, entendido éste como la suma del coste de ensamblaje de cada una de las piezas. 
  Implementa mediante ramificación y poda una solución a este problema con tal de encontrar el listado ordenado de piezas cuyo coste total de ensamblaje sea mínimo.
  </p>
  </div>
    </div>


<div class="alert alert-danger" style="width:80%; margin:0 auto; padding">
<center><p><h3> Código </h3></p> </center> 

<p>
<h3>INPUT</h3>
<ul>
<li>C: Matriz de costes de tamaño $M\times M$, donde M es el número de piezas. El elemento $i,j$ de la matriz representa el coste de ensamblaje para la pieza $i$ tras haber colocado $j$ piezas. 
</li>
</ul>
<br>
<h3>OUTPUT</h3>
Lista de números de $1$ a $M$ ordenados de forma que el coste total de ensamblaje sea mínimo. Ej: [5, 3, 2, 4, 1]
</p>

</div>

In [9]:
def branch_ensemble(C):
    """
    Función que obtiene el orden óptimo 
    de las piezas tal que el coste total es mínimo.
    
    Parameters
    ----------
        C, matriz de costes M x M
        
    Returns
    -------
        list
    """
    
    items = []
    
    return items

In [10]:
from random import uniform

# Número de piezas

M = 10 

# Matriz de costes generada aleatoriamente

C = {}

for i in range(M):
    C[i] = {}
    
    for j in range(M):
        C[i][j] = uniform(1, 25)
        
solucion = branch_ensemble(C)
print(solucion)

[]


### En esta sección debéis responder a las preguntas siguientes:

#### 1. Explica cómo has realizado tu implementación, enfatizando en la heurística empleada.

asdf

#### 2. Análisis de complejidad de la implementación

asdf